# Tensorboard WIP

This note book does not work.

In [1]:
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Data Loading, Preprocessing and Exploration

### Dataset Loading and Splitting

In [3]:
from read_fashion_mnist import FashionMnistLoader
## Windows users:
#from read_fashion_mnist_win import FashionMnistLoader

In [4]:
data_loader = FashionMnistLoader().get_all_data()

X_train_dev, X_test, y_train_dev, y_test = data_loader.standard_split()
X_train_dev.shape, X_test.shape, len(y_train_dev), len(y_test)

((60000, 784), (10000, 784), 60000, 10000)

In [5]:
X_train, X_dev, y_train, y_dev = data_loader.train_split(1/6)
X_train.shape, X_dev.shape, len(y_train), len(y_dev)

((50000, 784), (10000, 784), 50000, 10000)

In [6]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class FashionMnist(Dataset):
    
    def __init__(self, X, y):
        self.data = torch.from_numpy(X).float()
        self.target = torch.from_numpy(y).long()
        
    def __len__(self):
        return len(self.target)
        
    def __getitem__(self, index):
        img, tar = self.data[index], self.target[index]
        img = (img/255).reshape(-1, 1, 28, 28).squeeze()
        return img, tar
    
train_dataset = FashionMnist(X_train, y_train)
dev_dataset = FashionMnist(X_dev, y_dev)

In [7]:
classes = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
           'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']
num_classes = len(classes)

## Let's define Neural Network with pytorch

In [8]:
from torch import nn
from torch import optim
import torch.nn.functional as F

In [9]:
input_dim = 28 * 28

class LinearNN(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 10)
        
    def forward(self, x):
        x = x.reshape(x.size(0), -1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x
    
model = LinearNN()


In [10]:
criterion = nn.CrossEntropyLoss()

In [11]:
batch_size = 16

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

## In test set we put all dataset.
test_loader = torch.utils.data.DataLoader(dev_dataset, batch_size=len(dev_dataset), shuffle=True)

In [12]:
correct = 0
total = 0

for images, labels in test_loader:

    outputs = model(images)

    _, predicted = torch.max(outputs.data, 1)

    total += labels.size(0)
    correct += (predicted == labels).sum()

accuracy = 100.0 * correct.item() / total

print(accuracy,"%")

9.4 %


## Let's train it

In [13]:
learning_rate = 0.01
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [14]:
def train(n_epochs):
    
    loss_over_time = [] # to track the loss as the network trains
    
    for epoch in range(n_epochs):  # loop over the dataset multiple times
        
        running_loss = 0.0
        
        for batch_i, data in enumerate(train_loader):
            # get the input images and their corresponding labels
            inputs, labels = data

            # zero the parameter (weight) gradients
            optimizer.zero_grad()

            # forward pass to get outputs
            outputs = model(inputs)

            # calculate the loss
            loss = criterion(outputs, labels)

            # backward pass to calculate the parameter gradients
            loss.backward()

            # update the parameters
            optimizer.step()

            # print loss statistics
            # to convert loss into a scalar and add it to running_loss, we use .item()
            running_loss += loss.item()
            
            if batch_i % 1000 == 999:    # print every 1000 batches
                avg_loss = running_loss/1000
                # record and print the avg loss over the 1000 batches
                loss_over_time.append(avg_loss)
                print('Epoch: {}, Batch: {}, Avg. Loss: {}'.format(epoch + 1, batch_i+1, avg_loss))
                running_loss = 0.0

    print('Finished Training')
    return loss_over_time


In [15]:
n_epochs = 2 # start small to see if your model works, initially
model.train() # put model in train mode (important with, for example, dropouts, batch normalizations...)

training_loss = train(n_epochs) # call train and record the loss over time

Epoch: 1, Batch: 1000, Avg. Loss: 1.1365112312734127
Epoch: 1, Batch: 2000, Avg. Loss: 0.6690540061891079
Epoch: 1, Batch: 3000, Avg. Loss: 0.5833766202032566
Epoch: 2, Batch: 1000, Avg. Loss: 0.5308824920654297
Epoch: 2, Batch: 2000, Avg. Loss: 0.5050999495983124
Epoch: 2, Batch: 3000, Avg. Loss: 0.4953785157650709
Finished Training


In [16]:
# initialize tensor and lists to monitor test loss and accuracy
test_loss = torch.zeros(1)
class_correct = list(0. for i in range(num_classes))
class_total = list(0. for i in range(num_classes))

# set the module to evaluation mode
model.eval()

# get the input images and their corresponding labels
dataiter = iter(test_loader)
inputs, labels = dataiter.next()

# forward pass to get outputs
outputs = model(inputs)

# calculate the loss
test_loss = criterion(outputs, labels)

# get the predicted class from the maximum value in the output-list of class scores
_, predicted = torch.max(outputs.data, 1)

# compare predictions to true label
# this creates a `correct` Tensor that holds the number of correctly classified images in a batch
correct = np.squeeze(predicted.eq(labels.data.view_as(predicted)))

# calculate test accuracy for *each* object class
# we get the scalar value of correct items for a class, by calling `correct[i].item()`
for i in range(len(inputs)):
    label = labels.data[i]
    class_correct[label] += correct[i].item()
    class_total[label] += 1

print('Test Loss: {:.6f}\n'.format(test_loss.detach().numpy()))

Test Loss: 0.474515



In [17]:
for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

        
print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Accuracy of T-shirt/top: 78% (814/1034)
Test Accuracy of Trouser: 96% (965/1005)
Test Accuracy of Pullover: 72% (721/1001)
Test Accuracy of Dress: 84% (829/979)
Test Accuracy of  Coat: 86% (826/959)
Test Accuracy of Sandal: 91% (914/1004)
Test Accuracy of Shirt: 47% (481/1007)
Test Accuracy of Sneaker: 87% (880/1009)
Test Accuracy of   Bag: 93% (960/1027)
Test Accuracy of Ankle boot: 93% (915/975)

Test Accuracy (Overall): 83% (8305/10000)


# Keeping Track of Trainning with Tensorboard
Next, we implement a MNIST classifier using a simple neural network and visualize the training process using TensorBoard. In training phase, we plot the loss and accuracy functions through scalar_summary. In addition, we visualize the weight and gradient values of the parameters of the neural network using histogram_summary. PyTorch code for handling these summary functions can be found here.

In [29]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
#from logger import Logger
import logging


# create logger with 'spam_application'
logger = logging.getLogger('spam_application')
logger.setLevel(logging.DEBUG)
# create file handler which logs even debug messages
fh = logging.FileHandler('./logs2')

In [30]:
### Load dataset and DataLoader
# Fashion MNIST dataset 
## Your code here

# Data loader
data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Fully connected neural network model
model = LinearNN().to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=0.00001)  

In [31]:

data_iter = iter(data_loader)
iter_per_epoch = len(data_loader)
total_step = 50000

# Start training
for step in range(total_step):
    
    # Reset the data_iter
    if (step+1) % iter_per_epoch == 0:
        data_iter = iter(data_loader)

    # Fetch images and labels
    images, labels = next(data_iter)
    images, labels = images.view(images.size(0), -1).to(device), labels.to(device)
    
    # Forward pass
    outputs = model(images)
    loss = criterion(outputs, labels)
    
    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Compute accuracy
    _, argmax = torch.max(outputs, 1)
    accuracy = (labels == argmax.squeeze()).float().mean()

    if (step+1) % 500 == 0:
        print ('Step [{}/{}], Loss: {:.4f}, Acc: {:.2f}' 
               .format(step+1, total_step, loss.item(), accuracy.item()))
    if (step+1) % 100 == 0:
       # print ('Step [{}/{}], Loss: {:.4f}, Acc: {:.2f}' 
       #        .format(step+1, total_step, loss.item(), accuracy.item()))

        # ================================================================== #
        #                        Tensorboard Logging                         #
        # ================================================================== #

        # 1. Log scalar values (scalar summary)
        info = { 'loss': loss.item(), 'accuracy': accuracy.item() }

        for tag, value in info.items():
            logger.scalar_summary(tag, value, step+1)

        # 2. Log values and gradients of the parameters (histogram summary)
        for tag, value in model.named_parameters():
            tag = tag.replace('.', '/')
            logger.histo_summary(tag, value.data.cpu().numpy(), step+1)
            logger.histo_summary(tag+'/grad', value.grad.data.cpu().numpy(), step+1)


AttributeError: 'Logger' object has no attribute 'scalar_summary'

To run the TensorBoard, open a new terminal and run the command below. Then, open http://localhost:6006/ on your web browser.

```$ tensorboard --logdir='./logs' --port=6006```

In [ ]:
! tensorboard --logdir='./logs' --port=6006

In [ ]:
batch_size = 16
def train_reg(n_epochs):
    
    loss_over_time = [] # to track the loss as the network trains
    
    for epoch in range(n_epochs):  # loop over the dataset multiple times
        
        running_loss = 0.0
        
        for batch_i, data in enumerate(train_loader):
            # get the input images and their corresponding labels
            inputs, labels = data

            # zero the parameter (weight) gradients
            optimizer.zero_grad()

            # forward pass to get outputs
            outputs = model(inputs)

            # calculate the loss
            loss = criterion(outputs, labels)
            reg_loss = L1Regularizer(model).regularized_all_param(loss)
            # backward pass to calculate the parameter gradients
            reg_loss.backward()

            # update the parameters
            optimizer.step()

            # print loss statistics
            # to convert loss into a scalar and add it to running_loss, we use .item()
            running_loss += reg_loss.item()
            
            if batch_i % 1000 == 999:    # print every 1000 batches
                avg_loss = running_loss/1000
                # record and print the avg loss over the 1000 batches
                loss_over_time.append(avg_loss)
                print('Epoch: {}, Batch: {}, Avg. Loss: {}'.format(epoch + 1, batch_i+1, avg_loss))
                running_loss = 0.0

    print('Finished Training')
    return loss_over_time


In [ ]:
model = LinearNN()
n_epochs = 2 # start small to see if your model works, initially

model.train() # put model in train mode (important with, for example, dropouts, batch normalizations...)

training_loss = train_reg(n_epochs) # call train and record the loss over time

In [ ]:
# initialize tensor and lists to monitor test loss and accuracy
test_loss = torch.zeros(1)
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

# set the module to evaluation mode
model.eval()

for batch_i, data in enumerate(test_loader):
    
    # get the input images and their corresponding labels
    inputs, labels = data

    # forward pass to get outputs
    outputs = model(inputs)

    # calculate the loss
    loss = criterion(outputs, labels)
            
    # update average test loss 
    test_loss = test_loss + ((torch.ones(1) / (batch_i + 1)) * (loss.data - test_loss))
    
    # get the predicted class from the maximum value in the output-list of class scores
    _, predicted = torch.max(outputs.data, 1)
    
    # compare predictions to true label
    # this creates a `correct` Tensor that holds the number of correctly classified images in a batch
    correct = np.squeeze(predicted.eq(labels.data.view_as(predicted)))
    
    # calculate test accuracy for *each* object class
    # we get the scalar value of correct items for a class, by calling `correct[i].item()`
    for i in range(batch_size):
        label = labels.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

print('Test Loss: {:.6f}\n'.format(test_loss.numpy()[0]))

In [ ]:
for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

        
print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))